In [3]:
from google.colab import files

# Upload file
uploaded = files.upload()

Saving binary_classification_ds.csv to binary_classification_ds.csv


In [4]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [5]:
import pandas as pd
df=pd.read_csv("binary_classification_ds.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 104 entries, id to target
dtypes: float64(102), int64(2)
memory usage: 39.7 MB


In [ ]:
df.head()

,id,col_24,col_263,col_264,col_214,col_39,col_280,col_140,col_328,col_309,...,col_217,col_262,col_53,col_17,col_259,col_247,col_287,col_990,col_991,target
0,484858,0.998527,0.0,0.0,0.0,0.250000,0.0,0.011236,0.0,0.0,...,0.0,0.0,0.0625,0.153846,0.0,0.0,0.0,16.0,-1.0,0
1,241714,0.999966,0.0,0.0,0.0,0.191176,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0625,0.076923,0.0,0.0,1.0,12.0,-1.0,0
2,198733,0.998017,0.0,1.0,0.0,0.176471,0.0,0.112360,0.0,0.0,...,0.0,0.0,0.1875,0.230769,0.0,0.0,0.0,11.0,-1.0,0
3,106045,0.000000,0.0,0.0,0.0,0.102941,0.0,0.022472,0.0,0.0,...,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,6.0,-1.0,0
4,28242,0.997506,0.0,0.0,0.0,0.058824,0.0,0.011236,0.0,0.0,...,0.0,0.0,0.0625,0.153846,0.0,0.0,0.0,3.0,-1.0,0


In [6]:
df['target'].value_counts()

,count
target,
0,48347
1,1653


In [7]:
df.isnull().sum()

,0
id,0
col_24,0
col_263,0
col_264,0
col_214,0
...,...
col_247,0
col_287,0
col_990,0
col_991,0


In [8]:
def cap_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower_bound, upper_bound)

# Apply outlier capping to all columns except 'id' and 'target'
columns_to_check = df.drop(columns=['id', 'target']).columns
for col in columns_to_check:
    cap_outliers(df, col)

In [ ]:
print(cap_outliers,columns_to_check)

<function cap_outliers at 0x7c7b07107600> Index(['col_24', 'col_263', 'col_264', 'col_214', 'col_39', 'col_280',
       'col_140', 'col_328', 'col_309', 'col_201',
       ...
       'col_257', 'col_217', 'col_262', 'col_53', 'col_17', 'col_259',
       'col_247', 'col_287', 'col_990', 'col_991'],
      dtype='object', length=102)


In [9]:
X = df.drop(columns=["id", "target"])
y = df["target"]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [11]:
#handle class imbalance
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve ,auc

In [17]:
models={
    "Logisitic Regression":LogisticRegression(),
    "Decision Tree":DecisionTreeClassifier(),
    "Random Forest":RandomForestClassifier(),
    "Gradient Boost":GradientBoostingClassifier(),
    "Adaboost":AdaBoostClassifier()
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train_resampled, y_train_resampled) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training set performance
    model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
    model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score
    model_train_precision = precision_score(y_train, y_train_pred) # Calculate Precision
    model_train_recall = recall_score(y_train, y_train_pred) # Calculate Recall
    model_train_rocauc_score = roc_auc_score(y_train, y_train_pred)



    # Test set performance
    model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
    model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score
    model_test_precision = precision_score(y_test, y_test_pred) # Calculate Precision
    model_test_recall = recall_score(y_test, y_test_pred) # Calculate Recall
    model_test_rocauc_score = roc_auc_score(y_test, y_test_pred) #Calculate Roc


    print(list(models.keys())[i])
    print('Model performance for Training set')
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print('- F1 score: {:.4f}'.format(model_train_f1))

    print('- Precision: {:.4f}'.format(model_train_precision))
    print('- Recall: {:.4f}'.format(model_train_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_train_rocauc_score))



    print('----------------------------------')

    print('Model performance for Test set')
    print('- Accuracy: {:.4f}'.format(model_test_accuracy))
    print('- F1 score: {:.4f}'.format(model_test_f1))
    print('- Precision: {:.4f}'.format(model_test_precision))
    print('- Recall: {:.4f}'.format(model_test_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_test_rocauc_score))


    print('='*35)
    print('\n')

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logisitic Regression
Model performance for Training set
- Accuracy: 0.5988
- F1 score: 0.7209
- Precision: 0.0509
- Recall: 0.6309
- Roc Auc Score: 0.6143
----------------------------------
Model performance for Test set
- Accuracy: 0.5999
- F1 score: 0.7217
- Precision: 0.0521
- Recall: 0.6452
- Roc Auc Score: 0.6217


Decision Tree
Model performance for Training set
- Accuracy: 0.9992
- F1 score: 0.9992
- Precision: 0.9930
- Recall: 0.9818
- Roc Auc Score: 0.9908
----------------------------------
Model performance for Test set
- Accuracy: 0.9247
- F1 score: 0.9310
- Precision: 0.0498
- Recall: 0.0706
- Roc Auc Score: 0.5123


Random Forest
Model performance for Training set
- Accuracy: 0.9992
- F1 score: 0.9992
- Precision: 0.9930
- Recall: 0.9818
- Roc Auc Score: 0.9908
----------------------------------
Model performance for Test set
- Accuracy: 0.9647
- F1 score: 0.9497
- Precision: 0.0286
- Recall: 0.0020
- Roc Auc Score: 0.4998


Gradient Boost
Model performance for Training se

In [18]:
import lightgbm as lgb
from sklearn.metrics import classification_report, roc_auc_score

# Train LightGBM model
lgb_model = lgb.LGBMClassifier(random_state=42, class_weight="balanced")
lgb_model.fit(X_train_resampled, y_train_resampled)

# Evaluate the model
y_pred = lgb_model.predict(X_test)
y_pred_prob = lgb_model.predict_proba(X_test)[:, 1]

print("Classification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_prob))


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 33843, number of negative: 33843
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020576 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4335
[LightGBM] [Info] Number of data points in the train set: 67686, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     14504
           1       0.00      0.00      0.00       496

    accuracy                           0.97     15000
   macro avg       0.48      0.50      0.49     15000
weighted avg       0.93      0.97      0.95     15000

ROC AUC Score: 0.6524469612387238


In [20]:
# Training performance
y_train_pred = lgb_model.predict(X_train)
y_train_prob = lgb_model.predict_proba(X_train)[:, 1]
print("Training Classification Report:\n", classification_report(y_train, y_train_pred))
print("Training ROC AUC Score:", roc_auc_score(y_train, y_train_prob))

Training Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     33843
           1       1.00      0.01      0.03      1157

    accuracy                           0.97     35000
   macro avg       0.98      0.51      0.51     35000
weighted avg       0.97      0.97      0.95     35000

Training ROC AUC Score: 0.8395775438829833
